### Carrega Lista de Processos, e baixa as decisões


In [24]:
# Importando as Bibliotecas
import pandas as pd
import requests 
from bs4 import BeautifulSoup
import json
import pymongo
import yaml
from datetime import datetime

In [25]:
# Carrega Configurações do arquivo config
with open("config.yaml", "r") as ymlfile:
    cfg = yaml.full_load(ymlfile)

user = cfg["mongodb"]["user"]
passwd = cfg["mongodb"]["passwd"]
host = cfg["mongodb"]["host"]
dbmongo = cfg["mongodb"]["db"]


In [26]:
# Cria Conexão com banco
url_conexao = "mongodb+srv://" + user + ":" + passwd + "@" + host + "/?retryWrites=true&w=majority"
client = pymongo.MongoClient(url_conexao)
db = client[dbmongo]

In [27]:
# Coleções Mongo
col_processos = db['processo']
col_decisoes = db['decisoes']

In [28]:
dados = pd.DataFrame(list(col_processos.find({'processo.cod_empresa':{'$in':['ALLIANZ','SULAMERICA','TOKIOMARINE']}})))
df_processos = pd.json_normalize(dados['processo'])
df_processos.shape

(10464, 7)

In [29]:
def pega_valores(soup,tag):
    valor = soup.select_one('strong:-soup-contains("'+ tag +'")')
    if valor: 
        valor = valor.parent.text.strip()
        valor = valor.replace(tag +':','').replace('\t','').replace('\n','').strip()
        return valor

In [30]:
procs = df_processos['nro_processo']
len(procs)


10464

In [31]:
dados = []

for proc in procs:
    url = "https://esaj.tjsp.jus.br/cjpg/pesquisar.do?conversationId=&dadosConsulta.pesquisaLivre=&tipoNumero=UNIFICADO&numeroDigitoAnoUnificado=1003031-42.2022&foroNumeroUnificado=0001&dadosConsulta.nuProcesso=" + proc
    
    try: 
        html = requests.get(url).content
    except: 
        pass
    soup = BeautifulSoup(html, 'html.parser')

    aviso = soup.select_one('div:-soup-contains("Não foi encontrado nenhum")')

    if aviso: 
        print("Processo: " + proc + " sem resultados")
        
    else:
        print("Processo: " + proc + " com resultados")
        classe = pega_valores(soup,'Classe')
        assunto = pega_valores(soup,'Assunto')
        magistrado = pega_valores(soup,'Magistrado')
        comarca = pega_valores(soup,'Comarca')
        foro = pega_valores(soup,'Foro')
        vara = pega_valores(soup,'Vara')
        data_disp = pega_valores(soup,'Data de Disponibilização')
        texto_decisao = soup.find('div',attrs={'align':'justify','style':'display: none;'}).text.strip()

        dados.append([proc,classe,assunto,magistrado,comarca,foro,vara,data_disp,texto_decisao])


Processo: 1089728-02.2018.8.26.0100 sem resultados
Processo: 1006870-72.2022.8.26.0196 sem resultados
Processo: 1062946-87.2020.8.26.0002 com resultados
Processo: 1110295-20.2019.8.26.0100 com resultados
Processo: 1023520-68.2020.8.26.0002 com resultados
Processo: 1062549-62.2019.8.26.0002 sem resultados
Processo: 0007703-56.2012.8.26.0003 com resultados
Processo: 1005322-45.2018.8.26.0004 sem resultados
Processo: 1014533-42.2017.8.26.0004 sem resultados
Processo: 1014250-24.2014.8.26.0004 com resultados
Processo: 1013059-28.2020.8.26.0005 sem resultados
Processo: 0042387-98.2012.8.26.0005 com resultados
Processo: 1015116-79.2017.8.26.0019 com resultados
Processo: 1004945-02.2020.8.26.0361 com resultados
Processo: 1007847-61.2021.8.26.0564 com resultados
Processo: 1028352-79.2022.8.26.0001 sem resultados
Processo: 1028047-95.2022.8.26.0001 sem resultados
Processo: 1027254-59.2022.8.26.0001 sem resultados
Processo: 1025998-81.2022.8.26.0001 sem resultados
Processo: 0012164-28.2022.8.26.

In [32]:
df_decisoes = pd.DataFrame(dados,columns=["nro_processo","classe","assunto","magistrado","comarca","foro","vara","data_disp","texto_decisao"])
df_decisoes['instancia'] = "1ª Instância"


In [33]:
def carrega_decisao(linha):
    json_str = str(linha.to_json())
    json_str = '{"decisao":' + json_str + '}'
    col_decisoes.find_one_and_update({'decisao.nro_processo':linha['nro_processo']},{'$set':json.loads(json_str)},upsert=True)

In [34]:
for index,decisao in df_decisoes.iterrows(): 
    carrega_decisao(df_decisoes.loc[index])


In [35]:
termino = datetime.now().strftime("%d/%m/%y %H:%M:%S")
print("Acabou em: " + termino)

Acabou em: 23/09/22 12:25:09


In [ ]:
print("----")